# NSL export to CoLDP: vascular plants

In [1]:
import pandas as pd
import numpy as np

namefile = 'data/tracheophytes/APNI-names-2023-06-24-2725.csv'
taxonfile = 'data/tracheophytes/APC-taxon-2023-06-24-2757.csv'

### Names

In [11]:
df_names = pd.read_csv(namefile)

# Remove excess columns
columns = [
    'scientificNameID', 
    'nameAccordingToID', 
    'taxonRank', 
    'scientificName', 
    'genericName', 
    'specificEpithet', 
    'infraspecificEpithet', 
    'scientificNameAuthorship', 
    'nomenclaturalStatus',
    'originalNameUsageID', 
    'originalNameUsage'
]

df1 = df_names[columns]

# Instance ID is in nameAccordingToID for reasons unfathomable; rename to taxonID
df1 = df1.rename(columns={'nameAccordingToID': 'taxonID'})

# Filter for names with originalNameUsage
# Replace instance IDs in originalNameUsageID with scientificNameIDs; we will not need taxonID anymore after that
df2 = df1[~df1['originalNameUsageID'].isna()].merge(df1, how='left', left_on='originalNameUsageID', right_on='taxonID')[[
    'scientificNameID_x', 
    'taxonRank_x', 
    'scientificName_x', 
    'genericName_x', 
    'specificEpithet_x', 
    'infraspecificEpithet_x', 
    'scientificNameAuthorship_x', 
    'nomenclaturalStatus_x', 
    'scientificNameID_y', 
    'scientificName_y'
]]

column_mappings = {
    'scientificNameID_x': 'scientificNameID',
    'scientificName_x': 'scientificName',
    'taxonRank_x': 'taxonRank',
    'genericName_x': 'genericName',
    'specificEpithet_x': 'specificEpithet',
    'infraspecificEpithet_x': 'infraspecificEpithet',
    'scientificNameAuthorship_x': 'scientificNameAuthorship',
    'nomenclaturalStatus_x': 'nomenclaturalStatus',
    'scientificNameID_y': 'originalNameUsageID',
    'scientificName_y': 'originalNameUsage'
}

df2 = df2.rename(columns=column_mappings)

# Filter for names without originalNameUsage
df3 = df_names[df_names['originalNameUsage'].isna()][columns]
df3.drop(columns=['nameAccordingToID'], inplace=True)

# Merge dataframes
df5 = pd.concat([df2, df3])
df5 = df5.sort_values(by='scientificName')

# Make taxonRank lowercase
df5['taxonRank'] = df5['taxonRank'].str.lower()

# Map Latin rank names to less wankerish English ones
ranks = {
    'genus': 'genus',
    'species': 'species',
    'familia': 'family',
    'subfamilia': 'subfamily',
    'tribus': 'tribe',
    'subtribus': 'subtribe',
    'ordo': 'order',
    'subordo': 'suborder',
    'superordo': 'superorder',
    'subspecies': 'subspecies',
    'classis': 'class',
    'subclassis': 'subclass',
    'subdivision': 'subphylum',
    'subbdivision': 'subphylum',
    'varietas': 'variety',
    'nothovarietas': 'nothovariety',
    'subvarietas': 'subvariety',
    'subgenus': 'subgenus',
    'superspecies': 'superspecies',
    'forma': 'form',
    'subforma': 'subform',
    'division': 'phylum',
    'regnum': 'kingdom',
    'special form': 'special form',
    'sectio': 'section',
    'subsectio': 'subsection',
    'series': 'series',
    'subseries': 'subseries',
    'regio': 'domain',
    '[unknown]': '[unknown]',
    '[unranked]': '[unranked]',
    '[infragenus]': '[infragenus]',
    '[infraspecies]': '[infraspecies]'
}

taxon_ranks = []
uninomials = []
generic_names = []
for index, row in df5.iterrows():
    taxon_ranks.append(ranks[row['taxonRank']])

    uninomial = np.NaN
    generic_name = row['genericName']

    if not isinstance(row['genericName'], str):
        if not isinstance(row['scientificNameAuthorship'], str):
            uninomial = row['scientificName']
        else:
            uninomial = row['scientificName'][0:len(row['scientificName'])-len(row['scientificNameAuthorship'])-1]

    if row['taxonRank'] == 'genus':
        uninomial = row['genericName']
        generic_name = np.NaN

    uninomials.append(uninomial)
    generic_names.append(generic_name)

df5['taxonRank'] = taxon_ranks
df5['genericName'] = generic_names
df5['uninomial'] = uninomials

# Add nomenclaturalCode; 'ICN' in GBIF vocab. but 'botanical' in CoLDP
df5['code'] = 'botanical'

df5 = df5[[
    'scientificNameID', 
    'taxonRank', 
    'scientificName', 
    'uninomial',
    'genericName', 
    'specificEpithet', 
    'infraspecificEpithet', 
    'scientificNameAuthorship', 
    'code', 
    'nomenclaturalStatus', 
    'originalNameUsageID', 
    'originalNameUsage'
  ]]

df_nam = df5
df5

/tmp/ipykernel_337595/1792341983.py:1: DtypeWarning: Columns (27,28,29,30,31,35,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df_names = pd.read_csv(namefile)


,scientificNameID,taxonRank,scientificName,uninomial,genericName,specificEpithet,infraspecificEpithet,scientificNameAuthorship,code,nomenclaturalStatus,originalNameUsageID,originalNameUsage
85109,https://id.biodiversity.org.au/name/apni/51342601,species,(Dendrobium pugioniforme A.Cunn. x Dendrobium ...,NaN,Dendrobium,pugioniforme x striolatum x pugioniforme,NaN,NaN,botanical,[n/a],NaN,NaN
86018,https://id.biodiversity.org.au/name/apni/51342600,species,(Dockrillia pugioniformis (A.Cunn.) Rauschert ...,NaN,Dockrillia,pugioniformis x striolata x pugioniformis,NaN,NaN,botanical,[n/a],NaN,NaN
73342,https://id.biodiversity.org.au/name/apni/51264324,species,(Eucalyptus globulus Labill. x Eucalyptus urni...,NaN,Eucalyptus,globulus x urnigera x johnstonii,NaN,NaN,botanical,[n/a],NaN,NaN
73341,https://id.biodiversity.org.au/name/apni/233841,species,(Eucalyptus globulus Labill. x Eucalyptus urni...,NaN,Eucalyptus,globulus x urnigera x johnstonii,NaN,NaN,botanical,NaN,NaN,NaN
73581,https://id.biodiversity.org.au/name/apni/51264326,species,(Eucalyptus johnstonii Maiden x Eucalyptus glo...,NaN,Eucalyptus,johnstonii x globulus x urnigera,NaN,NaN,botanical,[n/a],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
17647,https://id.biodiversity.org.au/name/apni/176231,species,x Vappaculum x lavarackianum (M.A.Clem.) M.A.C...,NaN,Vappaculum,lavarackianum,NaN,(M.A.Clem.) M.A.Clem. & D.L.Jones,botanical,NaN,https://id.biodiversity.org.au/name/apni/122956,Dendrobium x lavarackianum M.A.Clem.
17648,https://id.biodiversity.org.au/name/apni/176232,species,x Vappaculum x superbiens (Rchb.f.) M.A.Clem. ...,NaN,Vappaculum,superbiens,NaN,(Rchb.f.) M.A.Clem. & D.L.Jones,botanical,NaN,https://id.biodiversity.org.au/name/apni/86587,Dendrobium x superbiens Rchb.f.
17649,https://id.biodiversity.org.au/name/apni/176233,species,x Vappaculum x vinicolor (St.Cloud) M.A.Clem. ...,NaN,Vappaculum,vinicolor,NaN,(St.Cloud) M.A.Clem. & D.L.Jones,botanical,NaN,https://id.biodiversity.org.au/name/apni/86859,Dendrobium x vinicolor St.Cloud
126908,https://id.biodiversity.org.au/name/apni/51727054,genus,xAbacopterella T.E.Almeida & A.R.Field,xAbacopterella,NaN,NaN,NaN,T.E.Almeida & A.R.Field,botanical,NaN,NaN,NaN


### Name relations

In [12]:
df_name_relationships = df5[~df5['originalNameUsageID'].isna()][['scientificNameID', 'scientificName', 'originalNameUsageID', 'originalNameUsage']]

df_namerel = df_name_relationships[['scientificNameID', 'originalNameUsageID']]
df_namerel['type'] = 'basionym'
df_namerel.rename(columns={'scientificNameID': 'nameID', 'originalNameUsageID': 'relatedNameID'}, inplace=True)

# Remove originalNameUsage columns from Names
drop_columns = [
    'originalNameUsageID',
    'originalNameUsage'
]

rename_columns = {
    'scientificNameID': 'ID',
    'taxonRank': 'rank',
    'scientificName': 'scientificName',
    'genericName': 'genus',
    'scientificNameAuthorship': 'authorship',
    'nomenclaturalStatus': 'status'
}

df5.drop(columns=drop_columns, inplace=True)
df5.rename(columns=rename_columns, inplace=True)

/tmp/ipykernel_337595/3173867388.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_namerel['type'] = 'basionym'
/tmp/ipykernel_337595/3173867388.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_namerel.rename(columns={'scientificNameID': 'nameID', 'originalNameUsageID': 'relatedNameID'}, inplace=True)


### Taxa

In [13]:
# Read Taxon export into dataframe
df_taxa = pd.read_csv(taxonfile)
df_taxa = df_taxa[df_taxa['nameType'] == 'scientific']

# Create dataframe with IDs
# This is used later to replace tree element IDs (in taxonID) with instance IDs (in taxonConceptID)
df_id = df_taxa[['taxonID', 'taxonConceptID']]

In [14]:
# Filter on accepted names; these are the taxa
df_tax = df_taxa[df_taxa['taxonomicStatus'] == 'accepted']

# Replace tree element IDs in taxonID with instance IDs (see above)
df_tax['taxonID'] = df_tax['taxonConceptID']

# Replace tree element IDs in parentNameUsageID with instance IDs
df_tax = df_tax.merge(df_id, how='left', left_on='parentNameUsageID', right_on='taxonID')
df_tax['parentNameUsageID'] = df_tax['taxonConceptID_y']
df_tax.drop(columns=['taxonID_y', 'taxonConceptID_y'], inplace=True)
df_tax.rename(columns={'taxonID_x': 'taxonID', 'taxonConceptID_x': 'taxonConceptID'}, inplace=True)

df_tax = df_tax[['taxonID',
 'scientificNameID',
 'scientificName',
 'scientificNameAuthorship',
 'nameAccordingTo',
 'nameAccordingToID',
 'parentNameUsageID',
 'taxonRank',
 'taxonRankSortOrder',
 'kingdom',
 'class',
 'subclass',
 'family',
 'taxonConceptID',
 'taxonRemarks',
 'higherClassification'
]]

# Translate ranks into English
df_tax['taxonRank'] = df_tax['taxonRank'].str.lower()
ranks = {
    'genus': 'genus',
    'species': 'species',
    'familia': 'family',
    'subfamilia': 'subfamily',
    'ordo': 'order',
    'subordo': 'suborder',
    'superordo': 'superorder',
    'subspecies': 'subspecies',
    'classis': 'class',
    'subclassis': 'subclass',
    'subdivision': 'subphylum',
    'subbdivision': 'subphylum',
    'varietas': 'variety',
    'subgenus': 'subgenus',
    'superspecies': 'superspecies',
    'forma': 'form',
    'division': 'phylum',
    'regnum': 'kingdom',
    'special form': 'special form',
    'sectio': 'section',
    'regio': 'domain',
    '[unknown]': '[unknown]'
}

taxon_ranks = []
for index, row in df_tax.iterrows():
    taxon_ranks.append(ranks[row['taxonRank']])

df_tax['taxonRank'] = taxon_ranks


/tmp/ipykernel_337595/2562605175.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tax['taxonID'] = df_tax['taxonConceptID']


#### Higher classification

In [15]:
# Create dictionary with higher taxa from instance ID
def get_higher_taxa(id, higher={}):
    row = df_tax.loc[df_tax['taxonID'] == id]

    if len(row) > 0:
        parent = row[[
            'taxonRank', 
            'scientificName', 
            'scientificNameAuthorship', 
            'parentNameUsageID']].to_dict(orient='records')[0]
        
        if isinstance(parent['scientificNameAuthorship'], str):
            higher[parent['taxonRank']] = parent['scientificName'][0:len(parent['scientificName'])-len(parent['scientificNameAuthorship'])-1]
        else:
            higher[parent['taxonRank']] = parent['scientificName']

        if isinstance(parent['parentNameUsageID'], str):
            return get_higher_taxa(parent['parentNameUsageID'], higher)
        else:
            return higher
    else:
        return higher

In [16]:
# Create higher classification
def create_higher_classification(id):
    higher = get_higher_taxa(id, {})
    keys = list(higher.keys())

    hcl = {}

    hcl['taxonID'] = id
    hcl['species'] = higher['species'] if 'species' in keys else np.NaN
    hcl['section'] = higher['section'] if 'section' in keys else np.NaN
    hcl['subgenus'] = higher['subgenus'] if 'subgenus' in keys else np.NaN
    hcl['genus'] = higher['genus'] if 'genus' in keys else np.NaN
    hcl['subtribe'] = higher['subtribe'] if 'subtribe' in keys else np.NaN
    hcl['tribe'] = higher['tribe'] if 'tribe' in keys else np.NaN
    hcl['subfamily'] = higher['subfamily'] if 'subfamily' in keys else np.NaN
    hcl['family'] = higher['family'] if 'family' in keys else np.NaN
    hcl['superfamily'] = higher['superfamily'] if 'superfamily' in keys else np.NaN
    hcl['suborder'] = higher['suborder'] if 'suborder' in keys else np.NaN
    hcl['order'] = higher['order'] if 'order' in keys else np.NaN
    hcl['subclass'] = higher['subclass'] if 'subclass' in keys else np.NaN
    hcl['class'] = higher['class'] if 'class' in keys else np.NaN
    hcl['subphylum'] = higher['subphylum'] if 'subphylum' in keys else np.NaN
    hcl['phylum'] = higher['phylum'] if 'phylum' in keys else np.NaN
    hcl['kingdom'] = higher['kingdom'] if 'kingdom' in keys else np.NaN

    cl = list(higher.values())
    for i in range(len(cl) // 2):
        cl[i], cl[-1 - i] = cl[-1 - i], cl[i]

    hcl['classification'] = ' | '.join(cl)

    return hcl

create_higher_classification('https://id.biodiversity.org.au/instance/lichen/30043034')


{'taxonID': 'https://id.biodiversity.org.au/instance/lichen/30043034',
 'species': nan,
 'section': nan,
 'subgenus': nan,
 'genus': nan,
 'subtribe': nan,
 'tribe': nan,
 'subfamily': nan,
 'family': nan,
 'superfamily': nan,
 'suborder': nan,
 'order': nan,
 'subclass': nan,
 'class': nan,
 'subphylum': nan,
 'phylum': nan,
 'kingdom': nan,
 'classification': ''}

In [17]:
cl = []
for index, row in df_tax.iterrows():
    cl.append(create_higher_classification(row['taxonID']))

df_higher = pd.DataFrame.from_dict(cl)

df_higher = df_higher.merge(df_tax[['taxonID', 'scientificName', 'taxonRank']], how='left', left_on='taxonID', right_on='taxonID')

df_higher = df_higher[['taxonID',
 'scientificName',
 'taxonRank',
 'kingdom',
 'phylum',
 'subphylum',
 'class',
 'subclass',
 'order',
 'suborder',
 'superfamily',
 'family',
 'tribe',
 'subfamily',
 'subtribe',
 'genus',
 'subgenus',
 'section',
 'species',
 'classification']]

df_higher.rename(columns={'classification': 'higherClassification'}, inplace=True)

df_higher

,taxonID,scientificName,taxonRank,kingdom,phylum,subphylum,class,subclass,order,suborder,superfamily,family,tribe,subfamily,subtribe,genus,subgenus,section,species,higherClassification
0,https://id.biodiversity.org.au/instance/apni/7...,Plantae Haeckel,kingdom,Plantae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plantae
1,https://id.biodiversity.org.au/instance/apni/6...,Charophyta Sachs,phylum,Plantae,Charophyta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plantae | Charophyta
2,https://id.biodiversity.org.au/instance/apni/6...,Equisetopsida C.Agardh,class,Plantae,Charophyta,NaN,Equisetopsida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plantae | Charophyta | Equisetopsida
3,https://id.biodiversity.org.au/instance/apni/6...,Cycadidae Pax,subclass,Plantae,Charophyta,NaN,Equisetopsida,Cycadidae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plantae | Charophyta | Equisetopsida | Cycadidae
4,https://id.biodiversity.org.au/instance/apni/6...,Cycadales Pers. ex Bercht. & J.Presl,order,Plantae,Charophyta,NaN,Equisetopsida,Cycadidae,Cycadales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plantae | Charophyta | Equisetopsida | Cycadid...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29994,https://id.biodiversity.org.au/instance/apni/5...,Tmesipteris norfolkensis P.S.Green,species,Plantae,Charophyta,NaN,Equisetopsida,Psilotidae,Psilotales,NaN,NaN,Psilotaceae,NaN,NaN,NaN,Tmesipteris,NaN,NaN,Tmesipteris norfolkensis,Plantae | Charophyta | Equisetopsida | Psiloti...
29995,https://id.biodiversity.org.au/instance/apni/6...,Tmesipteris obliqua Chinnock,species,Plantae,Charophyta,NaN,Equisetopsida,Psilotidae,Psilotales,NaN,NaN,Psilotaceae,NaN,NaN,NaN,Tmesipteris,NaN,NaN,Tmesipteris obliqua,Plantae | Charophyta | Equisetopsida | Psiloti...
29996,https://id.biodiversity.org.au/instance/apni/5...,Tmesipteris ovata N.A.Wakef.,species,Plantae,Charophyta,NaN,Equisetopsida,Psilotidae,Psilotales,NaN,NaN,Psilotaceae,NaN,NaN,NaN,Tmesipteris,NaN,NaN,Tmesipteris ovata,Plantae | Charophyta | Equisetopsida | Psiloti...
29997,https://id.biodiversity.org.au/instance/apni/5...,Tmesipteris parva N.A.Wakef.,species,Plantae,Charophyta,NaN,Equisetopsida,Psilotidae,Psilotales,NaN,NaN,Psilotaceae,NaN,NaN,NaN,Tmesipteris,NaN,NaN,Tmesipteris parva,Plantae | Charophyta | Equisetopsida | Psiloti...


#### Synonyms

In [18]:
# Filter on synonyms
df_syn = df_taxa[df_taxa['taxonomicStatus'].isin(['synonym', 'nomenclatural synonym', 'taxonomic synonym']) & ~df_taxa['acceptedNameUsageID'].isna()][[
    'taxonID',
    'scientificNameID',
    'scientificName',
    'acceptedNameUsageID',
    'acceptedNameUsage',
    'taxonomicStatus'
]]

# Replace tree element IDs in acceptedNameUsageID with instance IDs
df_syn = df_syn.merge(df_id, how='left', left_on='acceptedNameUsageID', right_on='taxonID')

df_syn.drop(columns=['acceptedNameUsageID', 'taxonID_y'], inplace=True)
df_syn.rename(columns={
    'taxonID_x': 'ID',
    'taxonConceptID': 'taxonID',
    'scientificNameID': 'nameID'    
}, inplace=True)
df_syn = df_syn[['ID', 'taxonID', 'nameID', 'scientificName', 'acceptedNameUsage', 'taxonomicStatus']]

# Get accepted names
df_dwc_syn = df_syn.merge(df_tax[['taxonID', 'scientificName']], left_on='taxonID', right_on='taxonID')

df_dwc_syn.drop(columns=['taxonID', 'nameID', 'acceptedNameUsage'], inplace=True)

df_dwc_syn.rename(columns={
    'ID': 'taxonID',
    'scientificName_x': 'scientificName',
    'scientificName_y': 'acceptedNameUsage'
}, inplace=True)

df_syn.drop(columns=['scientificName', 'acceptedNameUsage', 'taxonomicStatus'], inplace=True)

df_syn

,ID,taxonID,nameID
0,https://id.biodiversity.org.au/instance/apni/8...,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/223964
1,https://id.biodiversity.org.au/instance/apni/8...,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/225443
2,https://id.biodiversity.org.au/instance/apni/8...,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/225444
3,https://id.biodiversity.org.au/instance/apni/8...,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/225445
4,https://id.biodiversity.org.au/instance/apni/8...,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/225441
...,...,...,...
46721,https://id.biodiversity.org.au/instance/apni/8...,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/80218
46722,https://id.biodiversity.org.au/instance/apni/8...,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/96997
46723,https://id.biodiversity.org.au/instance/apni/8...,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/205867
46724,https://id.biodiversity.org.au/instance/apni/8...,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/88243


#### Remove excess columns

In [19]:
df_tax = df_tax[['taxonID',
 'scientificNameID',
 'nameAccordingToID',
 'parentNameUsageID',
 'taxonRemarks']]

df_tax.rename(columns={
    'taxonID': 'ID',
    'scientificNameID': 'nameID',
    'nameAccordingToID': 'accordingToID',
    'parentNameUsageID': 'parentID',
    'taxonRemarks': 'remarks'
}, inplace=True)

df_tax

,ID,nameID,accordingToID,parentID,remarks
0,https://id.biodiversity.org.au/instance/apni/7...,https://id.biodiversity.org.au/name/apni/54717,https://id.biodiversity.org.au/reference/apni/...,NaN,NaN
1,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/224706,https://id.biodiversity.org.au/reference/apni/...,https://id.biodiversity.org.au/instance/apni/7...,NaN
2,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/223519,https://id.biodiversity.org.au/reference/apni/...,https://id.biodiversity.org.au/instance/apni/6...,NaN
3,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/225054,https://id.biodiversity.org.au/reference/apni/...,https://id.biodiversity.org.au/instance/apni/6...,NaN
4,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/215527,https://id.biodiversity.org.au/reference/apni/...,https://id.biodiversity.org.au/instance/apni/6...,NaN
...,...,...,...,...,...
29994,https://id.biodiversity.org.au/instance/apni/5...,https://id.biodiversity.org.au/name/apni/125883,https://id.biodiversity.org.au/reference/apni/...,https://id.biodiversity.org.au/instance/apni/5...,NaN
29995,https://id.biodiversity.org.au/instance/apni/6...,https://id.biodiversity.org.au/name/apni/119030,https://id.biodiversity.org.au/reference/apni/...,https://id.biodiversity.org.au/instance/apni/5...,The name Tmesipteris tannensis var. normalis D...
29996,https://id.biodiversity.org.au/instance/apni/5...,https://id.biodiversity.org.au/name/apni/96852,https://id.biodiversity.org.au/reference/apni/...,https://id.biodiversity.org.au/instance/apni/5...,The name Tmesipteris tannensis var. normalis D...
29997,https://id.biodiversity.org.au/instance/apni/5...,https://id.biodiversity.org.au/name/apni/96879,https://id.biodiversity.org.au/reference/apni/...,https://id.biodiversity.org.au/instance/apni/5...,The name Tmesipteris tannensis var. normalis D...


### Reference

In [20]:
df_ref = df_taxa[['nameAccordingToID', 'nameAccordingTo']]
df_ref.drop_duplicates(inplace=True)
df_ref.rename(columns={'nameAccordingToID': 'ID', 'nameAccordingTo': 'citation'}, inplace=True)
df_ref

/tmp/ipykernel_337595/3187620901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ref.drop_duplicates(inplace=True)
/tmp/ipykernel_337595/3187620901.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ref.rename(columns={'nameAccordingToID': 'ID', 'nameAccordingTo': 'citation'}, inplace=True)


,ID,citation
0,https://id.biodiversity.org.au/reference/apni/...,"CHAH (2012), Australian Plant Census"
1,https://id.biodiversity.org.au/reference/apni/...,"CHAH (2009), Australian Plant Census"
3,https://id.biodiversity.org.au/reference/apni/...,"CHAH (2010), Australian Plant Census"
13,https://id.biodiversity.org.au/reference/apni/...,"CHAH (2008), Australian Plant Census"
14,https://id.biodiversity.org.au/reference/apni/...,"CHAH (2006), Australian Plant Census"
...,...,...
109531,https://id.biodiversity.org.au/reference/apni/...,"Chinnock, R.J. in McCarthy, P.M. (ed.) (1998),..."
109544,https://id.biodiversity.org.au/reference/apni/...,"McCarthy, P.M. in McCarthy, P.M. (ed.) (1998),..."
109570,https://id.biodiversity.org.au/reference/apni/...,"Chinnock, R.J. in McCarthy, P.M. (ed.) (1998),..."
109580,https://id.biodiversity.org.au/reference/apni/...,"Chinnock, R.J. in McCarthy, P.M. (ed.) (1998),..."


### Create CoLDP

In [ ]:
import os
from zipfile import ZipFile

os.chdir('/home/niels/code/jupyter-notebooks/nsl_export/coldp/tracheophytes')

df_tax.to_csv('taxon.tsv', sep='\t', index=False)
df_syn.to_csv('synonym.tsv', sep='\t', index=False)
df_nam.to_csv('name.tsv', sep='\t', index=False)
df_name_relationships.to_csv('dwc_basionyms.tsv', sep='\t', index=False)
df_namerel.to_csv('namerelation.tsv', sep='\t', index=False)
df_higher.to_csv('dwc_higherclassification.tsv', sep='\t', index=False)
df_dwc_syn.to_csv('dwc_synonym.tsv', sep='\t', index=False)
df_ref.to_csv('reference.tsv', sep='\t', index='False')

with ZipFile('nsl_tracheophytes_coldp.zip', 'w') as zipobj:
    zipobj.write('taxon.tsv')
    zipobj.write('name.tsv')
    zipobj.write('synonym.tsv')
    zipobj.write('namerelation.tsv')
    zipobj.write('reference.tsv')

os.chdir('/home/niels/code/jupyter-notebooks/nsl_export')